# Benchy

Example [benchy](https://build123d.readthedocs.io/en/latest/examples_1.html#examples-benchy)

Downloaded low_poly_benchy.stl using wget:
```sh
wget -O low_poly_benchy.stl https://github.com/gumyr/build123d/blob/dev/examples/low_poly_benchy.stl\?raw\=true
```

In [ ]:
from build123d import (
    Mesher,
)
from ocp_vscode import show
importer = Mesher()
benchy_stl = importer.read("low_poly_benchy.stl")

show(benchy_stl)